# Some useful Pandas functions

In [76]:
import pandas as pd
import json

1. [Get_prices_for_heaviest_item](#get_prices_for_heaviest_item)
2. [Reshape_temperature_data](#reshape_temperature_data)
3. [Compute_events_matrix_count](#compute_events_matrix_count)
4. [Diff_in_days](#diff_in_days)
5. [Return_location](#return_location)
6. [Return_post_codes](#Return_post_codes)

### 1. Get_prices_for_heaviest_item
<a id='get_prices_for_heaviest_item'></a>

In [77]:
def get_prices_for_heaviest_item(inventory):
    """
    The function takes a pandas.DataFrame with four column: "category", "price", "weight", "in_stock"
    and returns a pandas.Series containing the price of the heaviest weight per category of items still in stock.
    No items in the same category have the same weight to make things simpler to reason about.
    The returned Series should not have an index name and the values should be sorted in descending order.
    You should return an empty Series if there are not items in stock for all categories.

    :param inventory: pandas.DataFrame with four column "category", "price", "weight", "in_stock"
    :return: a pandas.Series with the category as index and the selected prices in descending order

    """

    inventory = inventory.loc[inventory["in_stock"]]
    inventory = inventory.loc[inventory["weight"] == inventory.groupby("category")["weight"].transform("max")]
    return inventory.set_index("category")["price"].rename().sort_values(ascending=False)

In [78]:
#Test function

inventory_df = pd.DataFrame({'category':['electronics', 'health', 'electronics', 'books'], 'price':[400, 5, 300, 20],
               'weight':[740, 100, 6000, 300], 'in_stock': [False, False, True, True]})
inventory_df.head()

,category,price,weight,in_stock
0,electronics,400,740,False
1,health,5,100,False
2,electronics,300,6000,True
3,books,20,300,True


In [79]:
get_prices_for_heaviest_item(inventory_df)

category
electronics    300
books           20
dtype: int64

### 2. Reshape_temperature_data
<a id='reshape_temperature_data'></a>

In [80]:
def reshape_temperature_data(measurements):
    """
    The function takes a pandas.DataFrame with 7 columns:
     "location", 'Jan-2018', 'Feb-2018', 'Mar-2018', "April-2018", "May-2018", "June-2018".
    This DataFrame represents temperature measurements in the first two quarter of 2018 for a particular city.
    This function should return a new DataFrame containing three columns: "location", "Date", "Value"
    and where each row represents a measurement in particular location at a particular date.
    The returned pandas.DataFrame should sort the values by location first and then by temperature measurement.
    It should also drop any missing values and reset the index of the returned DataFrame.


    :param measurements: pandas.DataFrame with seven columns:
    "location", 'Jan-2018', 'Feb-2018', 'Mar-2018', "April-2018", "May-2018", "June-2018"
    :return: a pandas.DataFrame containing three columns "location", "Date", "Value" with a row
    for each temperature measurement in a given location. There should be no missing values.
    """

    return pd.melt(measurements, id_vars=["location"], var_name="date").dropna().sort_values(
        by=["location", "value"]).reset_index(drop=True)

In [81]:
## Test function
import numpy as np

measurements_df = pd.DataFrame({'location':['Brussels', 'Paris'], 'Jan-2018':[2,2], 'Feb-2018':[3,3],
                               'Mar-2018':[8,9], 'April-2018':[12,np.NaN], 'May-2018':[14,15], 'June-2018':[17,18]})      


measurements_df.head()

,location,Jan-2018,Feb-2018,Mar-2018,April-2018,May-2018,June-2018
0,Brussels,2,3,8,12.0,14,17
1,Paris,2,3,9,NaN,15,18


In [82]:
reshape_temperature_data(measurements_df)

,location,date,value
0,Brussels,Jan-2018,2.0
1,Brussels,Feb-2018,3.0
2,Brussels,Mar-2018,8.0
3,Brussels,April-2018,12.0
4,Brussels,May-2018,14.0
5,Brussels,June-2018,17.0
6,Paris,Jan-2018,2.0
7,Paris,Feb-2018,3.0
8,Paris,Mar-2018,9.0
9,Paris,May-2018,15.0


### 3. Compute_events_matrix_count
<a id='compute_events_matrix_count'></a>

In [83]:
def compute_events_matrix_count(events):
    """
    The function takes a pandas.DataFrame containing 2 columns representing web events for a user:
    "user_id" and "event".
    This function should return a new DataFrame where each event value becomes a new column in the returned DataFrame.
    We expect the columns (events) to be in alphabetical order.

    For each event value, it calculates the count of that particular event for each userid.
    Missing values are filled with 0.
    Effectively, this function calculates the number of occurrence for each event type (columns) for each user (rows).
    You should return an empty Series if the input DataFrame is empty.

    :param events: pandas.DataFrame with two columns: "user_id" and "event"
    :return: a pandas.DataFrame returning the number of occurrence for each event type (columns) for each user (rows).
    """
    if events.empty:
        return pd.Series()
    #
    # df = pd.DataFrame(events.groupby("user_id")["event"].value_counts()).unstack().fillna(0)
    # df.columns = df.columns.droplevel(0)
    # return df

    return events.pivot_table(index="user_id", columns="event", aggfunc="size", fill_value=0)

In [84]:
## Tes function

event_matrix = pd.DataFrame({'user_id':[1234, 4321, 1234, 1234, 4321,3456], 
                             'event':['click','click','click','play','play','pause']})

event_matrix.head()

,user_id,event
0,1234,click
1,4321,click
2,1234,click
3,1234,play
4,4321,play


In [85]:
compute_events_matrix_count(event_matrix)

event,click,pause,play
user_id,,,
1234,2,0,1
3456,0,1,0
4321,1,0,1


### 4. Diff_in_days
<a id='diff_in_days'></a>

In [86]:
from datetime import datetime
def diff_in_days(df):
    """
    The function takes a pandas DataFrame with two columns "time_1"
    and "time_2" of UNIX timestamps given in seconds (you will need to specify
    the unit if using pd.to_datetime).

    The function should return a new dataFrame with one single column
    "difference_days" consisting of the absolute difference in days between
    time_1 and time_2.

    
    :param df: DataFrame with the two columns of timestamps
    :return: new dataframe with differences in days between timestamps
    """
    df_convert = [df[col_df].apply(datetime.fromtimestamp) for col_df in df.columns]

    return pd.DataFrame(abs(df_convert[0] - df_convert[1]).dt.days, columns=["difference_days"])

In [87]:
## Test function

days_df = pd.DataFrame({'time_1': [1456694829, 1455845363], 'time_2': [1455845363, 1456694829]})

days_df.head()

,time_1,time_2
0,1456694829,1455845363
1,1455845363,1456694829


In [88]:
diff_in_days(days_df)

,difference_days
0,9
1,9


### 5. Return_location
<a id='return_location'></a>

In [89]:
def return_location(df):
    """
    The function takes a pandas DataFrame with one column, locations,
    containing information about a specific location. The info is stored in a string
    that can be loaded as a json object.
    The function should return a DataFrame with one column, "short_name" that contains the
    value associated with the key "short_name" for each row.

    :param df: DataFrame with the locations column
    :return: new DataFrame with the short_name column
    """

    return pd.DataFrame(df["locations"].apply(json.loads).apply(pd.Series)["short_name"], columns=["short_name"])

In [90]:
## Test function

locations_df = pd.DataFrame({'locations': ['{"short_name": "Detroit, MI", "id": 2391585}', 
                            '{"short_name": "Tracy, CA", "id": 2507550}']})
locations_df.head()

,locations
0,"{""short_name"": ""Detroit, MI"", ""id"": 2391585}"
1,"{""short_name"": ""Tracy, CA"", ""id"": 2507550}"


In [91]:
return_location(locations_df)

,short_name
0,"Detroit, MI"
1,"Tracy, CA"


### 6. Return_post_codes
<a id='Return_post_codes'></a>

In [92]:
def return_post_codes(df):
    """
    The function takes a pandas DataFrame with one column, text, that
    contains an arbitrary text. The function should extract all post-codes that
    appear in that text and concatenate them together with " | ". The result is
    a new dataframe with a column "postcodes" that contains all concatenated
    postcodes.

  
    :param df: a DataFrame with the text column
    :return: new DataFrame with the postcodes column
    """
    df = df.text.str.extractall(r"([a-zA-Z]{1,2}\d{1,2}[a-zA-Z]?\s*\d[a-zA-Z]{2})").reset_index()
    df_output = pd.DataFrame(df.groupby("level_0")[0].apply(" | ".join))
    df_output.columns = ["postcodes"]
    del df_output.index.name
    return df_output


In [93]:
## Test function

postcodes_intext_df = pd.DataFrame({'text': ['Great Doddington, Wellingborough NN29 7TA, UK\nTaylor, Leeds LS14 6JA, UK',
                                   'This is some text, and here is a postcode CB4 9NE']})

postcodes_intext_df.head()

,text
0,"Great Doddington, Wellingborough NN29 7TA, UK\..."
1,"This is some text, and here is a postcode CB4 9NE"


In [94]:
return_post_codes(postcodes_intext_df)

,postcodes
0,NN29 7TA | LS14 6JA
1,CB4 9NE
